In [1]:
import numpy as np
from sklearn.metrics import accuracy_score
#from  utilities import *
from tqdm import tqdm
import matplotlib.pyplot as plt
import sys
import glob
import h5py
import cv2

In [2]:
class ReseauNeuronesArtificiel():
    def _init__(self, W, b):
        self.W = W
        self.b = b
        self.learning_rate = None
        self.X_train = None
        self.y_train = None
        self.cout = None
    
    def __model(self, X):
        Z=X.dot(self.W) + self.b
        A=1/(1+np.exp(-Z))
        return A
    
    def __initialisation(self, X, y, learning_rate):
        self.X_train = X
        self.y_train = y
        self.W=np.random.randn(self.X_train.shape[1],1)
        self.b=np.random.randn(1)
        self.learning_rate = learning_rate
        self.cout = []
    
    def __logLoss(self, A):
        return 1/len(self.y_train)*np.sum(-self.y_train*np.log(A)-(1-self.y_train)*np.log(1-A))
    
    def __gradient(self, A):
        dW=1/len(self.y_train)*np.dot(self.X_train.T,A-self.y_train)
        db=1/len(self.y_train)*sum(A-self.y_train)
        return (dW,db)
    
    def __update(self,dW,db):
        self.W=self.W-self.learning_rate*dW
        self.b=self.b-self.learning_rate*db

    def fit(self, X,y, learning_rate=0.1, n_iter=10000):
        #initialisation
        self.__initialisation(X, y, learning_rate)
        
        for i in range(n_iter):
            A=self.__model(X,)
            self.cout.append(self.__logLoss(A))
            dW,db=self.__gradient(A)
            self.__update(dW,db)
        
        y_pred=self.predict(X)
        
        return y_pred
        
    def predict(self, X_predict):
        A=self.__model(X_predict)
        #print(A)
        return A>=0.5

In [17]:
IMG_WIDTH = 150
IMG_HEIGHT = 150

datarepo = "data/"
datarepocats = datarepo + "cats/"
datarepodogs = datarepo + "dogs/"

h5file = 'dataset/dataset1.hdf5'
nfiles = len(glob.glob(datarepocats+"*.jpg"))
print(f'count of image files nfiles={nfiles}')

#on load les images de chats
with h5py.File(h5file,'w') as  h5f:
    
    x = h5f.create_dataset('X',shape=(nfiles, IMG_WIDTH, IMG_HEIGHT, 3), dtype=int)
    y = h5f.create_dataset('y',shape=(nfiles, 1), dtype=int)
    for cnt, ifile in enumerate(glob.iglob(datarepocats+"*.jpg")) :
        img = cv2.imread(ifile, cv2.IMREAD_COLOR)
        # or use cv2.IMREAD_GRAYSCALE, cv2.IMREAD_UNCHANGED
        img_resize = cv2.resize( img, (IMG_WIDTH, IMG_HEIGHT) )
        x[cnt:cnt+1:,:,:] = img_resize
        y[cnt] = 1

count of image files nfiles=1011
0
(1011, 1)
1
(1011, 1)
2
(1011, 1)
3
(1011, 1)
4
(1011, 1)
5
(1011, 1)
6
(1011, 1)
7
(1011, 1)
8
(1011, 1)
9
(1011, 1)
10
(1011, 1)
11
(1011, 1)
12
(1011, 1)
13
(1011, 1)
14
(1011, 1)
15
(1011, 1)
16
(1011, 1)
17
(1011, 1)
18
(1011, 1)
19
(1011, 1)
20
(1011, 1)
21
(1011, 1)
22
(1011, 1)
23
(1011, 1)
24
(1011, 1)
25
(1011, 1)
26
(1011, 1)
27
(1011, 1)
28
(1011, 1)
29
(1011, 1)
30
(1011, 1)
31
(1011, 1)
32
(1011, 1)
33
(1011, 1)
34
(1011, 1)
35
(1011, 1)
36
(1011, 1)
37
(1011, 1)
38
(1011, 1)
39
(1011, 1)
40
(1011, 1)
41
(1011, 1)
42
(1011, 1)
43
(1011, 1)
44
(1011, 1)
45
(1011, 1)
46
(1011, 1)
47
(1011, 1)
48
(1011, 1)
49
(1011, 1)
50
(1011, 1)
51
(1011, 1)
52
(1011, 1)
53
(1011, 1)
54
(1011, 1)
55
(1011, 1)
56
(1011, 1)
57
(1011, 1)
58
(1011, 1)
59
(1011, 1)
60
(1011, 1)
61
(1011, 1)
62
(1011, 1)
63
(1011, 1)
64
(1011, 1)
65
(1011, 1)
66
(1011, 1)
67
(1011, 1)
68
(1011, 1)
69
(1011, 1)
70
(1011, 1)
71
(1011, 1)
72
(1011, 1)
73
(1011, 1)
74
(1011, 1)
75

In [19]:
dataset = "dataset/dataset1.hdf5"

train_dataset = h5py.File(dataset, "r")
X_train = np.array(train_dataset["X"][:]) # your train set features
y_train = np.array(train_dataset["y"]) # your train set labels
print(X_train.shape)
print(y_train.shape)
print(X_train)

(1011, 150, 150, 3)
(1011, 1)
[[[[ 19  18  24]
   [117 101 122]
   [ 96  96 104]
   ...
   [ 82  74  83]
   [ 85  80  86]
   [ 15  12  14]]

  [[ 17  17  22]
   [131 120 139]
   [142 144 146]
   ...
   [134 122 114]
   [102  95  93]
   [ 13  10  11]]

  [[ 15  26  30]
   [125 119 137]
   [248 253 250]
   ...
   [183 170 136]
   [ 96  87  76]
   [ 15  11  11]]

  ...

  [[ 22  15  24]
   [136 125 144]
   [255 254 252]
   ...
   [212 206 194]
   [ 92  88  84]
   [ 12  10  10]]

  [[ 26  21  23]
   [ 88  79  93]
   [ 80  77  75]
   ...
   [ 67  61  57]
   [ 67  66  62]
   [ 10   9   9]]

  [[  4   5   1]
   [  4   1   7]
   [  7   5   5]
   ...
   [  5   1   2]
   [  0   1   0]
   [  1   1   1]]]


 [[[ 97 117 122]
   [ 96 115 120]
   [ 98 115 120]
   ...
   [ 14  42 111]
   [ 11  40 107]
   [ 10  39 106]]

  [[ 99 119 124]
   [101 120 125]
   [102 119 125]
   ...
   [ 17  43 107]
   [ 16  42 103]
   [ 15  41 102]]

  [[ 94 114 119]
   [ 97 116 121]
   [101 118 123]
   ...
   [ 20  44 103

In [ ]:
"""plt.figure(figsize=(16,8))
for i in range(1,10):
    plt.subplot(4,5,i)
    plt.imshow(x_train[i],cmap='gray')
    plt.title (y_train[i])
    plt.tight_layout()
plt.show()"""